<a href="https://colab.research.google.com/github/Thabby07/richter-predictor/blob/master/TFM_Entrenamiento_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from datetime import datetime

In [0]:
train_values = pd.read_csv('/content/train_values.csv')
train_labels = pd.read_csv('/content/train_labels.csv')
test_values = pd.read_csv('/content/test_values.csv')

In [0]:
train = train_values.set_index('building_id').join(train_labels.set_index('building_id'), how='left')

In [53]:
# TODO Change string categorical to numeric categorical
# Now we exclude string categorical variables
train = train.select_dtypes(exclude=['object'])
test = test_values.select_dtypes(exclude=['object'])
train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,689827,13,1310,202,2,0,7,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,780104,20,136,4769,2,5,4,7,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,618103,8,463,6973,1,10,11,3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,333694,26,886,66,1,5,7,4,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,947549,5,1079,4274,2,35,13,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
damage_1 = train[train['damage_grade'] == 1].sample(n=25000)
damage_2 = train[train['damage_grade'] == 2].sample(n=25000)
damage_3 = train[train['damage_grade'] == 3].sample(n=25000)

In [40]:
train_sample = pd.concat([damage_1, damage_2, damage_3]).reset_index()
train_sample.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,689827,13,1310,202,2,0,7,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,780104,20,136,4769,2,5,4,7,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,618103,8,463,6973,1,10,11,3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,333694,26,886,66,1,5,7,4,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,947549,5,1079,4274,2,35,13,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [52]:
train.iloc[:, 30]

0        0
1        0
2        0
3        0
4        0
        ..
74995    0
74996    0
74997    0
74998    0
74999    0
Name: has_secondary_use_other, Length: 75000, dtype: int64

In [48]:
test.columns.size

31

In [0]:
train = train_sample

In [0]:
#X = train.iloc[:, 0:38].values
#y = train.iloc[:, 38].values
X = train.iloc[:, 0:30].values
y = train.iloc[:, 30].values

In [0]:
# Partition train-test & scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [0]:
classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(train.iloc[:, 0:31], train.iloc[:, 31])
y_pred = classifier.predict(test)

In [0]:
f1_micro_score = f1_score(y_test, y_pred, average='macro')
print(f1_micro_score)

In [0]:
test.reset_index(level=0, inplace=True)

In [0]:
output = pd.DataFrame({'building_id': test['building_id'], 
                       'damage_grade': y_pred})

timestamp = str(np.rint(datetime.timestamp(datetime.now())).astype(int))

path = "/content/" + timestamp + "_competition_result.csv"
output.to_csv(path_or_buf=path, index=False)